In [39]:
import re
import json

def parse_room_data(file_path):
    with open(file_path, "r") as file:
        lines = file.readlines()

    num_rooms = int(lines[0].strip())  
    index = 1
    rooms = {}
    all_coords = []

    for i in range(num_rooms):
       
        wall_count, floor_count, ceiling_count = map(int, lines[index].strip().split())
        index += 1

        room_key = f"room_{i + 1}"
        rooms[room_key] = {}

        def parse_surface(line):
            pattern = r"\(([-\d.]+), ([-\d.]+), ([-\d.]+)\)"
            return [tuple(map(float, match)) for match in re.findall(pattern, line)]

        # Wall
        for j in range(wall_count):
            coords = parse_surface(lines[index].strip())
            all_coords.extend(coords)
            rooms[room_key][f"{room_key}_wall_surface_{j + 1}"] = coords
            index += 1
            
        # Floor
        for _ in range(floor_count):
            coords = parse_surface(lines[index].strip())
            all_coords.extend(coords)
            rooms[room_key][f"{room_key}_ground_surface_1"] = coords
            index += 1

        # Ceiling
        for _ in range(ceiling_count):
            coords = parse_surface(lines[index].strip())
            all_coords.extend(coords)
            rooms[room_key][f"{room_key}_ceiling_surface_1"] = coords
            index += 1
            
    if all_coords:
        min_x = min(coord[0] for coord in all_coords)
        min_y = min(coord[1] for coord in all_coords)
        min_z = min(coord[2] for coord in all_coords)
        max_x = max(coord[0] for coord in all_coords)
        max_y = max(coord[1] for coord in all_coords)
        max_z = max(coord[2] for coord in all_coords)

        bounding_box = {
            "lowerCorner": (min_x, min_y, min_z),
            "upperCorner": (max_x, max_y, max_z)
        }
    else:
        bounding_box = {"lowerCorner": (0,0,0), "upperCorner": {30,30,30}}
        
    return rooms, bounding_box

pc_in =  "C:/Users/SEVEN/pcp_exercise/data/input data/" 
pc_out =  "C:/Users/SEVEN/pcp_exercise/data/output data/"  
file_path = pc_in + "rooms_data.txt"
room_data, bounding_box = parse_room_data(file_path)

In [3]:
from lxml import etree

# generate wall surface
def create_wall_surface(surface_id, surface_name, coords):
    surface = etree.Element("{http://www.opengis.net/citygml/building/2.0}WallSurface", gml_id=surface_id)
    name = etree.SubElement(surface, "{http://www.opengis.net/gml}name")
    name.text = surface_name
    
    lod2_multi_surface = etree.SubElement(surface, "{http://www.opengis.net/citygml/building/2.0}lod2MultiSurface")
    multi_surface = etree.SubElement(lod2_multi_surface, "{http://www.opengis.net/gml}MultiSurface")
    surface_member = etree.SubElement(multi_surface, "{http://www.opengis.net/gml}surfaceMember")
    polygon = etree.SubElement(surface_member, "{http://www.opengis.net/gml}Polygon", gml_id=f"Poly_{surface_id}")
    exterior = etree.SubElement(polygon, "{http://www.opengis.net/gml}exterior")
    
    linear_ring = etree.SubElement(exterior, "{http://www.opengis.net/gml}LinearRing", gml_id=f"Poly_{surface_id}_0")
    for coord in coords:
        pos = etree.SubElement(linear_ring, "{http://www.opengis.net/gml}pos")
        pos.text = " ".join(map(str, coord))
    
    return surface

# generate ground surface
def create_ground_surface(surface_id, surface_name, coords):
    surface = etree.Element("{http://www.opengis.net/citygml/building/2.0}GroundSurface", gml_id=surface_id)
    name = etree.SubElement(surface, "{http://www.opengis.net/gml}name")
    name.text = surface_name
    
    lod2_multi_surface = etree.SubElement(surface, "{http://www.opengis.net/citygml/building/2.0}lod2MultiSurface")
    multi_surface = etree.SubElement(lod2_multi_surface, "{http://www.opengis.net/gml}MultiSurface")
    surface_member = etree.SubElement(multi_surface, "{http://www.opengis.net/gml}surfaceMember")
    polygon = etree.SubElement(surface_member, "{http://www.opengis.net/gml}Polygon", gml_id=f"Poly_{surface_id}")
    exterior = etree.SubElement(polygon, "{http://www.opengis.net/gml}exterior")
    
    linear_ring = etree.SubElement(exterior, "{http://www.opengis.net/gml}LinearRing", gml_id=f"Poly_{surface_id}_0")
    for coord in coords:
        pos = etree.SubElement(linear_ring, "{http://www.opengis.net/gml}pos")
        pos.text = " ".join(map(str, coord))
    
    return surface

# CityGML structure
def create_citygml():
    namespaces = {
        "core": "http://www.opengis.net/citygml/2.0",
        "bldg": "http://www.opengis.net/citygml/building/2.0",
        "gml": "http://www.opengis.net/gml",
        "xsi": "http://www.w3.org/2001/XMLSchema-instance",
        "app": "http://www.opengis.net/citygml/appearance/2.0",
        "gen": "http://www.opengis.net/citygml/generics/2.0",
        "xAL": "urn:oasis:names:tc:ciq:xsdschema:xAL:2.0",
        "xlink": "http://www.w3.org/1999/xlink"
    }

    city_model = etree.Element("{http://www.opengis.net/citygml/2.0}CityModel", nsmap=namespaces)

    xsi_schema_location = "http://www.opengis.net/citygml/2.0 http://schemas.opengis.net/citygml/2.0/cityGMLBase.xsd "
    xsi_schema_location += "http://www.opengis.net/citygml/appearance/2.0 http://schemas.opengis.net/citygml/appearance/2.0/appearance.xsd "
    xsi_schema_location += "http://www.opengis.net/citygml/building/2.0 http://schemas.opengis.net/citygml/building/2.0/building.xsd "
    xsi_schema_location += "http://www.opengis.net/citygml/generics/2.0 http://schemas.opengis.net/citygml/generics/2.0/generics.xsd"
    city_model.set("{http://www.w3.org/2001/XMLSchema-instance}schemaLocation", xsi_schema_location)

    name = etree.SubElement(city_model, "{http://www.opengis.net/gml}name")
    name.text = "Test Box"

    bounded_by = etree.SubElement(city_model, "{http://www.opengis.net/gml}boundedBy")
    envelope = etree.SubElement(bounded_by, "{http://www.opengis.net/gml}Envelope", srsDimension="3")
    lower_corner = etree.SubElement(envelope, "{http://www.opengis.net/gml}lowerCorner", srsDimension="3")
    lower_corner.text = "0 0 0"
    upper_corner = etree.SubElement(envelope, "{http://www.opengis.net/gml}upperCorner", srsDimension="3")
    upper_corner.text = "15 15 5"

    city_object_member = etree.SubElement(city_model, "{http://www.opengis.net/citygml/2.0}cityObjectMember")
    building = etree.SubElement(city_object_member, "{http://www.opengis.net/citygml/building/2.0}Building", gml_id="box_1")
    
    # wall surface boundary points
    wall_surfaces_coords = custom_surfaces

    for surface_id, coords in wall_surfaces_coords.items():
        surface_name = f"Outer Wall {surface_id[-1]}"
        surface_element = create_wall_surface(surface_id, surface_name, coords)
        bounded_by = etree.SubElement(building, "{http://www.opengis.net/citygml/building/2.0}boundedBy")
        bounded_by.append(surface_element)
    
    # ground surface boundary points
    ground_coords = {
        "ground_surface_1": [(-1.3823, 1.8719, 0.00410563), (-1.3823, -2.7481, 0.00410563), (5.8177, -2.7481, 0.00410563), 
                (5.8177, 1.8719, 0.00410563), (3.4777, 1.8719, 0.00410563), (2.4877, 0.971899, 0.00410563), (2.2477, 1.0619, 0.00410563), 
                (1.3477, 1.8719, 0.00410563)],
        "ground_surface_2": [(5.8187, -2.93876, 0.00410563), (-1.3813, -2.93876, 0.00410563), (-1.3813, -11.8488, 0.00410563), 
                             (0.2387, -11.8488, 0.00410563), (0.2387, -4.40876, 0.00410563), (5.8187, -4.37876, 0.00410563)],
        "ground_surface_3": [(0.410451, -5.6171, 0.00410563), (0.410451, -9.8471, 0.00410563), (5.81045, -9.8471, 0.00410563), 
                             (5.81045, -5.6171, 0.00410563)]
    }
        
    for surface_id, coords in ground_coords.items():
        surface_name = f"Ground Surface {surface_id[-1]}"
        ground_surface = create_ground_surface(surface_id, surface_name, coords)
        bounded_by = etree.SubElement(building, "{http://www.opengis.net/citygml/building/2.0}boundedBy")
        bounded_by.append(ground_surface)
   

    tree = etree.ElementTree(city_model)
    return tree

def save_citygml(filename="3_rooms.gml"):
    tree = create_citygml()
    
    with open(filename, "wb") as file:
        tree.write(file, xml_declaration=True, encoding="UTF-8", pretty_print=True, doctype="<!DOCTYPE CityModel>")

save_citygml("3_rooms.gml")

In [40]:
print(bounding_box)

{'lowerCorner': (0.321, 0.123602, 0.219394), 'upperCorner': (13.3526, 32.5697, 3.32817)}


In [51]:
from lxml import etree

# generate surface function
def create_surface(surface_type, surface_id, surface_name, coords):
    surface = etree.Element(f"{{http://www.opengis.net/citygml/building/2.0}}{surface_type}", gml_id=surface_id)
    name = etree.SubElement(surface, "{http://www.opengis.net/gml}name")
    name.text = surface_name
    
    lod2_multi_surface = etree.SubElement(surface, "{http://www.opengis.net/citygml/building/2.0}lod2MultiSurface")
    multi_surface = etree.SubElement(lod2_multi_surface, "{http://www.opengis.net/gml}MultiSurface")
    surface_member = etree.SubElement(multi_surface, "{http://www.opengis.net/gml}surfaceMember")
    polygon = etree.SubElement(surface_member, "{http://www.opengis.net/gml}Polygon", gml_id=f"Poly_{surface_id}")
    exterior = etree.SubElement(polygon, "{http://www.opengis.net/gml}exterior")
    
    linear_ring = etree.SubElement(exterior, "{http://www.opengis.net/gml}LinearRing", gml_id=f"Poly_{surface_id}_0")
    for coord in coords:
        pos = etree.SubElement(linear_ring, "{http://www.opengis.net/gml}pos")
        pos.text = " ".join(map(str, coord))
    
    return surface

# generate room function
# def create_room(room_id, room_name, surfaces):
#     room = etree.Element("{http://www.opengis.net/citygml/building/2.0}Room", gml_id=room_id)
#     name = etree.SubElement(room, "{http://www.opengis.net/gml}name")
#     name.text = room_name
#     lod4_solid = etree.SubElement(room, "{http://www.opengis.net/citygml/building/2.0}lod4Solid")
#     solid = etree.SubElement(lod4_solid, "{http://www.opengis.net/gml}Solid")
#     exterior = etree.SubElement(solid, "{http://www.opengis.net/gml}exterior")
#     composite_surface = etree.SubElement(exterior, "{http://www.opengis.net/gml}CompositeSurface")
    
#     for surface_id in surfaces:
#         surface_member = etree.SubElement(composite_surface, "{http://www.opengis.net/gml}surfaceMember")
#         orientable_surface = etree.SubElement(surface_member, "{http://www.opengis.net/gml}OrientableSurface", orientation="-")
#         base_surface = etree.SubElement(orientable_surface, "{http://www.opengis.net/gml}baseSurface", 
#                                        attrib={"{http://www.w3.org/1999/xlink}href": f"#{surface_id}"})
        
#     for surface_id in surfaces:
#         surface_type = "WallSurface" if "wall" in surface_id else "RoofSurface" if "ceiling" in surface_id else "GroundSurface"
#         surface_element = create_surface(surface_type, surface_id, surface_id, surfaces[surface_id])
#         bounded_by = etree.SubElement(building, "{http://www.opengis.net/citygml/building/2.0}boundedBy")
#         bounded_by.append(surface_element)
    
#     return room

# main CityGML structure
def create_citygml(room_data):
    namespaces = {
        "core": "http://www.opengis.net/citygml/2.0",
        "bldg": "http://www.opengis.net/citygml/building/2.0",
        "gml": "http://www.opengis.net/gml",
        "xsi": "http://www.w3.org/2001/XMLSchema-instance",
        "app": "http://www.opengis.net/citygml/appearance/2.0",
        "gen": "http://www.opengis.net/citygml/generics/2.0",
        "xAL": "urn:oasis:names:tc:ciq:xsdschema:xAL:2.0",
        "xlink": "http://www.w3.org/1999/xlink"
    }

    city_model = etree.Element("{http://www.opengis.net/citygml/2.0}CityModel", nsmap=namespaces)

    xsi_schema_location = "http://www.opengis.net/citygml/2.0 http://schemas.opengis.net/citygml/2.0/cityGMLBase.xsd "
    xsi_schema_location += "http://www.opengis.net/citygml/appearance/2.0 http://schemas.opengis.net/citygml/appearance/2.0/appearance.xsd "
    xsi_schema_location += "http://www.opengis.net/citygml/building/2.0 http://schemas.opengis.net/citygml/building/2.0/building.xsd "
    xsi_schema_location += "http://www.opengis.net/citygml/generics/2.0 http://schemas.opengis.net/citygml/generics/2.0/generics.xsd"
    city_model.set("{http://www.w3.org/2001/XMLSchema-instance}schemaLocation", xsi_schema_location)

    name = etree.SubElement(city_model, "{http://www.opengis.net/gml}name")
    name.text = "Test Box"

    bounded_by = etree.SubElement(city_model, "{http://www.opengis.net/gml}boundedBy")
    envelope = etree.SubElement(bounded_by, "{http://www.opengis.net/gml}Envelope", srsDimension="3")
    lower_corner = etree.SubElement(envelope, "{http://www.opengis.net/gml}lowerCorner", srsDimension="3")
    lower_corner.text = "0.321 0.123602 0.219394"
    upper_corner = etree.SubElement(envelope, "{http://www.opengis.net/gml}upperCorner", srsDimension="3")
    upper_corner.text = "13.3526 32.5697 3.32817"

    city_object_member = etree.SubElement(city_model, "{http://www.opengis.net/citygml/2.0}cityObjectMember")
    building = etree.SubElement(city_object_member, "{http://www.opengis.net/citygml/building/2.0}Building", gml_id="box_1")
    
    
    for room_id, surfaces in room_data.items():
        # room_surfaces = []
        room = etree.Element("{http://www.opengis.net/citygml/building/2.0}Room", gml_id=room_id)
        name = etree.SubElement(room, "{http://www.opengis.net/gml}name")
        name.text = room_id
        lod4_solid = etree.SubElement(room, "{http://www.opengis.net/citygml/building/2.0}lod4Solid")
        solid = etree.SubElement(lod4_solid, "{http://www.opengis.net/gml}Solid")
        exterior = etree.SubElement(solid, "{http://www.opengis.net/gml}exterior")
        composite_surface = etree.SubElement(exterior, "{http://www.opengis.net/gml}CompositeSurface")

        for surface_id in surfaces:
            surface_member = etree.SubElement(composite_surface, "{http://www.opengis.net/gml}surfaceMember")
            orientable_surface = etree.SubElement(surface_member, "{http://www.opengis.net/gml}OrientableSurface", orientation="-")
            base_surface = etree.SubElement(orientable_surface, "{http://www.opengis.net/gml}baseSurface", 
                                           attrib={"{http://www.w3.org/1999/xlink}href": f"#{surface_id}"})

        for surface_id in surfaces:
            surface_type = "WallSurface" if "wall" in surface_id else "RoofSurface" if "ceiling" in surface_id else "GroundSurface"
            surface_element = create_surface(surface_type, surface_id, surface_id, surfaces[surface_id])
            bounded_by = etree.SubElement(room, "{http://www.opengis.net/citygml/building/2.0}boundedBy")
            bounded_by.append(surface_element)
        
        # for surface_id in surfaces:
        #     room_surfaces.append(f"Poly_{surface_id}")

        # room = create_room(room_id, room_id, surfaces)
        interior_room = etree.SubElement(building, "{http://www.opengis.net/citygml/building/2.0}interiorRoom")
        interior_room.append(room)
        
    tree = etree.ElementTree(city_model)
    return tree

# save GML file
def save_citygml(room_data,filename="room_structure.gml"):
    tree = create_citygml(room_data)
    with open(filename, "wb") as file:
        tree.write(file, xml_declaration=True, encoding="UTF-8", pretty_print=True)

save_citygml(room_data,pc_out + "room_structure.gml")